In [ ]:
import pandas as pd
import yfinance as yf
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

file_path = input('Enter the file path: ')
# Load the aggregated Smartly data
smartly_agg = pd.read_csv(file_path)


# Convert account names to uppercase for consistency (this ensures case-insensitive matching later)
smartly_agg['Account'] = smartly_agg['Account'].str.upper()

# Define a dictionary to map full country names to their corresponding market initials
country_to_market = {
    'AUSTRALIA': 'AU',
    'UNITED KINGDOM': 'UK',
    'POLAND': 'PL',
    'ITALY': 'IT',
    'FRANCE': 'FR',
    'SPAIN': 'ES',
    'GERMANY': 'DE',
    'CANADA': 'CA',
    'UNITED STATES': 'US',
}

# Function to normalize account names by extracting market initials
def normalize_account(account):
    account_upper = account.upper()  # Convert account name to uppercase
    # Iterate through each market initial and check if it is part of the account name
    for market in country_to_market.values():
        if market in account_upper:
            return market
    # If no market initials found, return 'Unknown'
    return 'Unknown'

# Apply the normalization function to the 'Account' column to extract the market initials
smartly_agg = smartly_agg.replace('PANDORA UNITED STATES', 'US')
smartly_agg['Market'] = smartly_agg['Account'].apply(normalize_account)

# Specific market mappings for Pandora accounts (special cases)
market_mapping = {
    'PANDORA POLAND': 'PL',
    'PANDORA IPROSPECT': 'UK',
    'PANDORA - GERMANY': 'DE',
    'PANDORA': 'CA',
    'PANDORA SPAIN': 'ES',
    'PANDORA_GB': 'UK',
    'PANDORA GERMANY': 'DE',
}

# For accounts with 'Unknown' market, apply the mapping to assign correct market
smartly_agg_rest = smartly_agg[smartly_agg['Market'] == 'Unknown']
smartly_agg_rest['Market'] = smartly_agg_rest['Account'].map(market_mapping)

# Combine the original dataframe with the updated 'smartly_agg_rest' containing the correct markets
smartly_agg = pd.concat([smartly_agg[smartly_agg['Market'] != 'Unknown'], smartly_agg_rest])

# Define the currency pairs for conversion (for downloading forex data)
currency_pairs = ['USDCAD=X', 'USDPLN=X', 'USDEUR=X', 'USDGBP=X', 'USDAUD=X']

# Download the forex data (1-day period to get the latest exchange rates)
data = yf.download(currency_pairs, period='1d', interval='1d')

# Extract the closing exchange rates from the downloaded data
exchange_rates = data['Close'].iloc[-1]

# Assign individual exchange rates for each currency pair
USDCAD = exchange_rates['USDCAD=X']
USDPLN = exchange_rates['USDPLN=X']
USDEUR = exchange_rates['USDEUR=X']
USDGBP = exchange_rates['USDGBP=X']
USDAUD = exchange_rates['USDAUD=X']

# Map of market codes to the corresponding exchange rates
map = {
    'DE': USDEUR,
    'UK': USDGBP,
    'PL': USDPLN,
    'CA': USDCAD,
    'ES': USDEUR,
    'IT': USDEUR,
    'FR': USDEUR,
    'US': 1,  # USD is already in local currency
    'AU': USDAUD,
}

# Filter rows where the Advertising Platform is 'tiktok'
smartly_agg_tiktok = smartly_agg[smartly_agg['Advertising Platform'] == 'tiktok']

# Apply the exchange rate map to convert 'Spend' into local currency for 'tiktok' platform
smartly_agg_tiktok['exchange_rate'] = smartly_agg_tiktok['Market'].map(map)
smartly_agg_tiktok['Spend_in_local_currency'] = smartly_agg_tiktok['Spend'] * smartly_agg_tiktok['exchange_rate']

# Update the main dataframe with the converted 'Spend_in_local_currency' for tiktok platform
smartly_agg.loc[smartly_agg['Advertising Platform'] == 'tiktok', 'Spend_in_local_currency'] = smartly_agg_tiktok['Spend_in_local_currency']

# For all other accounts, 'Spend' is assumed to already be in local currency (no conversion needed)
smartly_agg['Spend_in_local_currency'] = smartly_agg['Spend_in_local_currency'].fillna(smartly_agg['Spend'])


# Group by 'Market' and sum the 'Spend_in_local_currency' for each market
paid_social_smartly = smartly_agg.groupby(['Market'])['Spend_in_local_currency'].sum().round(2)

print(paid_social_smartly)


Enter the file path: /content/Weekly_Paid_Social.csv


[*********************100%***********************]  5 of 5 completed

Market
AU     73683.71
CA     45595.48
DE    138319.26
ES     33306.97
FR     58207.23
IT     79727.52
PL     74208.56
UK    138949.98
US    326628.00
Name: Spend_in_local_currency, dtype: float64


In [ ]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Authenticate and connect to Google Sheets
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/LiftLab/pandora-dashboard-data-0a402d52e1d4.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet
sheet = client.open('LL QA').worksheet('Sheet2')

# Assuming `paid_social_smartly` is a Series, we convert it to a DataFrame
# This will create a DataFrame with a single column
df = pd.DataFrame(paid_social_smartly)



# Convert DataFrame to list of lists for update
sheet.update([df.columns.values.tolist()] + df.values.tolist())

print("✅ Data successfully updated in Google Sheets!")


✅ Data successfully updated in Google Sheets!


In [ ]:
print(smartly_agg)

                              Account ID                  Account  \
0                      10156689910474867         PANDORA UK - GBP   
1                       1112706958882724               PANDORA PL   
2                        185771395657547               PANDORA CA   
3                       1970974156303010               PANDORA US   
4                        256177075058415               PANDORA AU   
6                        306212499969457               PANDORA IT   
7   329c1ede-fe44-4a11-b133-74c8d53a26b3           PANDORA CANADA   
8                        355425371665903               PANDORA ES   
9                        472795509876316               PANDORA DE   
11                          549756533137               PANDORA US   
12                          549756556001  PANDORA AUSTRALIA (AUD)   
13                          549756929517           PANDORA FRANCE   
15                          549759834852               PANDORA ES   
17                          549763

In [ ]:
from google.colab import drive
drive.mount('/content/drive')